In [1]:
import psycopg2

In [11]:
conn = psycopg2.connect(dbname='testdb', user='postgres', 
                        password='postgres', host='localhost')
cursor = conn.cursor()

In [12]:
cursor.execute("select current_database()")
db_name = cursor.fetchone()[0]

In [13]:
db_name

'testdb'

In [14]:
commands = (
        """
        CREATE TABLE vendors (
            vendor_id SERIAL PRIMARY KEY,
            vendor_name VARCHAR(255) NOT NULL
        )
        """,
        """ CREATE TABLE parts (
                part_id SERIAL PRIMARY KEY,
                part_name VARCHAR(255) NOT NULL
                )
        """,
        """
        CREATE TABLE part_drawings (
                part_id INTEGER PRIMARY KEY,
                file_extension VARCHAR(5) NOT NULL,
                drawing_data BYTEA NOT NULL,
                FOREIGN KEY (part_id)
                REFERENCES parts (part_id)
                ON UPDATE CASCADE ON DELETE CASCADE
        )
        """,
        """
        CREATE TABLE vendor_parts (
                vendor_id INTEGER NOT NULL,
                part_id INTEGER NOT NULL,
                PRIMARY KEY (vendor_id , part_id),
                FOREIGN KEY (vendor_id)
                    REFERENCES vendors (vendor_id)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (part_id)
                    REFERENCES parts (part_id)
                    ON UPDATE CASCADE ON DELETE CASCADE
        )
        """)

In [15]:
try:
    for command in commands:
        cursor.execute(command)
        # close communication with the PostgreSQL database server
    cursor.close()
    # commit the changes
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

In [16]:
conn

<connection object at 0x7f78bc33a528; dsn: 'password=xxx dbname=testdb user=postgres host=localhost', closed: 1>